In [10]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from itertools import combinations

In [11]:
#turns str glich name into a 1 or 0
def gliches_get_stiches(aList,glich):
    new_list = []
    for i in range(len(aList)):
        if aList[i] == glich:
            new_list += [1]
        else:
            new_list += [0]
    return np.array(new_list)

In [12]:
def all_da_gliches(aList):
    new_list = []
    
    labels = data['label'].unique()
    for i in range(len(aList)):
        zeros = np.zeros(22)
        for x in range(len(labels)):
            if aList[i] == labels[x]:
                zeros[x] = 1

        new_list += [zeros]
    return np.array(new_list)

In [13]:
data = pd.read_csv('trainingset_v1d1_metadata.csv')

#glich to test
glich = 'Whistle'

#what features do we want 
column_selection = ["duration","peak_frequency","central_freq","amplitude","snr","bandwidth"]
#["duration","peak_frequency","central_freq","snr"]



#training data
data_train = data[data['sample_type'] == 'train']
glich_list = data_train['label'].tolist()
y_train = all_da_gliches(glich_list)
x_train = np.array(data_train[column_selection])

#testing data
data_test = data[data['sample_type'] == 'test']
glich_list2 = data_test['label'].tolist()
y_test = all_da_gliches(glich_list2)
x_test = np.array(data_test[column_selection])

#validation data
data_val = data[data['sample_type'] == 'validation']
glich_list3 = data_val['label'].tolist()
y_val = all_da_gliches(glich_list3)
x_val = np.array(data_val[column_selection])
#print(data_test[column_selection])


In [16]:
def multi_model_trial(some_data):
    model3 = RandomForestClassifier(n_estimators= 100)
    keep = []
    feature_list = list(combinations(some_data,3)) + list(combinations(some_data,4)) + list(combinations(some_data,5))

    for i in feature_list:
        
        x_train = np.array(data_train[np.array(i)])
        scaler = StandardScaler().fit(x_train)
        standardized_X = scaler.transform(x_train)
        scaler2 = Normalizer().fit(standardized_X)
        std_and_norm_X_train = scaler2.transform(standardized_X)
        
        x_test = np.array(data_test[np.array(i)])
        standardized_X_test = scaler.transform(x_test)
        std_and_norm_X_test = scaler2.transform(standardized_X_test)

        #no change
        #model3.fit(x_train, y_train)
        #y_predict = model3.predict(x_test)
        
        #std 
        model3.fit(standardized_X, y_train)
        y_predict = model3.predict(standardized_X_test)
        
        #std and norm
        #model3.fit(std_and_norm_X_train, y_train)
        #y_predict = model3.predict(std_and_norm_X_test)

        keep += [y_predict]
    return keep
the_models2 = multi_model_trial(column_selection)
better_model2 = []
for i in the_models2:
       if metrics.accuracy_score(y_test, i) > .5:
              better_model2.append(i)
keep2 = []
for i in better_model2:
    keep2.append(metrics.accuracy_score(y_test, i))
print(keep2)
final2 = []
for i in range(len(better_model2[0])):
       keep2 = np.zeros(22)
       for x in range(len(better_model2)):
              keep2 += better_model2[x][i]
       for i in range(len(keep2)):
              if keep2[i] == max(keep2):
                     keep2[i] = 1
              else:
                     keep2[i] = 0
       final2.append(keep2)
final_array2 = np.array(final2)

print("Accuracy:",metrics.accuracy_score(y_test, final_array2))

[0.72264631043257, 0.7913486005089059, 0.7947413061916879, 0.72264631043257, 0.5733672603901612, 0.5733672603901612, 0.5207803223070399, 0.6497031382527566, 0.5665818490245971, 0.547921967769296, 0.726039016115352, 0.7150127226463104, 0.6293469041560644, 0.7633587786259542, 0.7319762510602206, 0.7302798982188295, 0.5581000848176421, 0.5224766751484309, 0.556403731976251, 0.811704834605598, 0.8269720101781171, 0.7480916030534351, 0.8278201865988125, 0.820186598812553, 0.8303647158608991, 0.712468193384224, 0.6276505513146734, 0.6174724342663274, 0.7065309584393554, 0.8040712468193384, 0.7659033078880407, 0.7616624257845632, 0.8015267175572519, 0.6039016115351993, 0.8269720101781171, 0.8057675996607294, 0.8176420695504665, 0.8278201865988125, 0.7099236641221374, 0.7938931297709924]
Accuracy: 0.5826972010178118


In [23]:
better_model2 = []
for i in the_models2:
       if metrics.accuracy_score(y_test, i) > .82:
              better_model2.append(i)
keep2 = []
for i in better_model2:
    keep2.append(metrics.accuracy_score(y_test, i))
print(keep2)
final2 = []
for i in range(len(better_model2[0])):
       keep2 = np.zeros(22)
       for x in range(len(better_model2)):
              keep2 += better_model2[x][i]
       for i in range(len(keep2)):
              if keep2[i] == max(keep2):
                     keep2[i] = 1
              else:
                     keep2[i] = 0
       final2.append(keep2)
final_array2 = np.array(final2)

print("Accuracy:",metrics.accuracy_score(y_test, final_array2))

[0.8269720101781171, 0.8278201865988125, 0.820186598812553, 0.8303647158608991, 0.8269720101781171, 0.8278201865988125]
Accuracy: 0.8481764206955047
